In [1]:
import os
import gradio as gr
from transformers import pipeline
from openai import OpenAI

# Configure a API Key da OpenAI
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [2]:
# Modelos Hugging Face
chatbot = pipeline("text-generation", model="gpt2")
image_captioner = pipeline("image-to-text", model="nlpconnect/vit-gpt2-image-captioning")
speech_to_text = pipeline("automatic-speech-recognition", model="openai/whisper-base")

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

C:\Users\gabri\anaconda3\envs\llms\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\gabri\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

...

normalizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

preprocessor_config.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [3]:
# Função de chat multimodal
def multimodal_chat(mensagem, imagem, audio, historico):
    resposta_texto = ""

    # Transcreve áudio
    if audio is not None:
        transcricao = speech_to_text(audio)["text"]
        mensagem = (mensagem or "") + " " + transcricao
        resposta_texto += f"(Transcrição de áudio: {transcricao})\n\n"

    # Gera legenda da imagem
    if imagem is not None:
        legenda = image_captioner(imagem)[0]["generated_text"]
        mensagem = (mensagem or "") + f" (imagem: {legenda})"
        resposta_texto += f"(Descrição da imagem: {legenda})\n\n"

    # Resposta do chatbot
    if mensagem:
        saida = chatbot(mensagem, max_length=100, num_return_sequences=1)[0]["generated_text"]
        resposta_texto += saida

    historico.append((mensagem, resposta_texto))
    return historico, historico

In [4]:
# Função de geração de imagem com OpenAI
def gerar_imagem(prompt):
    response = client.images.generate(
        model="gpt-4o-mini",  # ou "dall-e-3"
        prompt=prompt,
        size="512x512"
    )
    return response.data[0].url


In [5]:
# Interface Gradio
with gr.Blocks() as demo:
    gr.Markdown("# 🤖 Chat Multimodal + Geração de Imagens")

    chatbot_ui = gr.Chatbot()
    with gr.Row():
        texto = gr.Textbox(label="Mensagem de texto")
        imagem = gr.Image(label="Envie uma imagem", type="filepath")
        audio = gr.Audio(label="Envie áudio", type="filepath")
    enviar = gr.Button("Enviar")
    estado = gr.State([])

    enviar.click(fn=multimodal_chat,
                 inputs=[texto, imagem, audio, estado],
                 outputs=[chatbot_ui, estado])

    # Seção de geração de imagens
    gr.Markdown("## 🎨 Gerar Imagens com OpenAI")
    prompt_img = gr.Textbox(label="Prompt da Imagem")
    botao_img = gr.Button("Gerar Imagem")
    saida_img = gr.Image(label="Imagem Gerada")

    botao_img.click(fn=gerar_imagem, inputs=prompt_img, outputs=saida_img)

demo.launch()

C:\Users\gabri\AppData\Local\Temp\ipykernel_29408\3755360094.py:5: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot_ui = gr.Chatbot()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
